<a href="https://colab.research.google.com/github/vaish-8468/Machine_Learning/blob/main/ID3_Algo_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


Loading the  dataset

In [2]:
dataset=pd.read_csv('/content/assign1_dataset.csv')
dataset.head(14)

,Outlook,Temperature,Humidity,Wind,Decision
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [3]:

X=[feature for feature in dataset]
X.remove("Decision")


Defining a Node class

In [4]:
class Node:
  def __init__(self):
    self.children=[]
    self.value=""
    self.isLeaf=False
    self.pred=""

Defining a function Entropy

In [5]:
def Entropy(samples):
  positive=0.0
  negative=0.0
  for _, row in samples.iterrows():
    if row["Decision"]=="yes":
      positive+=1
    else:
      negative+=1
  if positive==0.0 or negative==0.0:
    return 0.0
  else:
    pos=positive/(positive+negative)
    neg=negative/(positive+negative)
    return -((pos* math.log(pos,2)) + (neg* math.log(neg,2)))


Defining an Information Gain function

In [6]:
def Information_Gain(samples,attributes):
  unique=np.unique(samples[attributes])
  Gain=Entropy(samples)
  for i in unique:
    subData=samples[samples[attributes]==i]
    sub_Entropy=Entropy(subData)
    Gain-=(float(len(subData))/float(len(samples)))*sub_Entropy
  return Gain  

In [7]:
check=np.unique(dataset["Decision"])
check

array(['no', 'yes'], dtype=object)

In [8]:
check=X.copy()
check

['Outlook', 'Temperature', 'Humidity', 'Wind']

ID3 function

In [9]:
def ID3(samples,attributes):
  root=Node()
  Max_feature=" "
  Max_Gain=0
  for feature in attributes:
    Gain=Information_Gain(samples,feature)
    if(Gain> Max_Gain):
      Max_Gain=Gain
      Max_feature=feature
  root.value=Max_feature
  unique=np.unique(samples[Max_feature])
  for i in unique:
    subData=samples[samples[Max_feature]==i]
    if Entropy(subData)==0.0:
      newNode=Node()
      newNode.isLeaf=True
      newNode.value=i
      newNode.pred= np.unique(subData["Decision"])   
      root.children.append(newNode)
    else:
      dummyNode = Node()
      dummyNode.value = i
      New_attributes = attributes.copy()
      New_attributes.remove(Max_feature)
      Child = ID3(subData, New_attributes)
      dummyNode.children.append(Child)
      root.children.append(dummyNode)

  return root

Function to draw the Decision Tree

In [10]:
def Print_Tree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for Child in root.children:
        Print_Tree(Child, depth + 1)

In [11]:
def Classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new sample", test," is:", child.pred)
                exit
            else:
                Classify (child.children[0], new)

In [12]:
root = ID3(dataset, X)
print("Decision Tree is:")
Print_Tree(root)




Decision Tree is:
Outlook
	overcast ->  ['yes']

	rain
		Wind
			strong ->  ['no']

			weak ->  ['yes']

	sunny
		Humidity
			high ->  ['no']

			normal ->  ['yes']



In [14]:
test = {"Outlook":"sunny", "Temperature":"mild", "Humidity":"high", "Wind":"weak"}
Classify (root, test)

Predicted Label for new sample {'Outlook': 'sunny', 'Temperature': 'mild', 'Humidity': 'high', 'Wind': 'weak'}  is: ['no']
